In [1]:
import nltk
import re
from gensim.models import Word2Vec
import multiprocessing

import io

In [2]:
#Tokenizer 다운로드
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\powerinfo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Data Collection

In [3]:
# Children's Book Test text 데이터 추출 => https://huggingface.co/datasets/cam-cst/cbt
'''
from datasets import load_dataset
import re

def text_regexp_processing(text):
    # 1. 괄호 및 괄호 안 내용 제거
    text = re.sub(r'\([^)]*\)', '', text)

    # 2. 축약어 ' 토큰 생성
    text = re.sub(r"'(\w)", r"APOSTROPHE\1", text)
    
    # 3. 마침표(.),문자,공백 을 제외한 특수문자 제거
    text = re.sub(r'[^a-zA-Z0-9.\s]', '', text)

    # 4. 축약어 토큰 -> ' 로 변경
    text = text.replace("APOSTROPHE", "'")
    
    return text

# 추출할 list 갯수 param
list_num = 20000
    
dataset = load_dataset("cbt", "CN")

texts = dataset['train']['sentences'][:list_num]

cleaned_texts = []
for text in texts:
    text = ' '.join(text)

    text = text_regexp_processing(text)
    
    cleaned_texts.append(text)

with open(f"cbt_clean_{list_num}_train.txt", "w", encoding="utf-8") as f:
    for line in cleaned_texts:
        f.write(line + " ")
'''

'\nfrom datasets import load_dataset\nimport re\n\ndef text_regexp_processing(text):\n    # 1. 괄호 및 괄호 안 내용 제거\n    text = re.sub(r\'\\([^)]*\\)\', \'\', text)\n\n    # 2. 축약어 \' 토큰 생성\n    text = re.sub(r"\'(\\w)", r"APOSTROPHE\x01", text)\n    \n    # 3. 마침표(.),문자,공백 을 제외한 특수문자 제거\n    text = re.sub(r\'[^a-zA-Z0-9.\\s]\', \'\', text)\n\n    # 4. 축약어 토큰 -> \' 로 변경\n    text = text.replace("APOSTROPHE", "\'")\n    \n    return text\n\n# 추출할 list 갯수 param\nlist_num = 20000\n    \ndataset = load_dataset("cbt", "CN")\n\ntexts = dataset[\'train\'][\'sentences\'][:list_num]\n\ncleaned_texts = []\nfor text in texts:\n    text = \' \'.join(text)\n\n    text = text_regexp_processing(text)\n    \n    cleaned_texts.append(text)\n\nwith open(f"cbt_clean_{list_num}_train.txt", "w", encoding="utf-8") as f:\n    for line in cleaned_texts:\n        f.write(line + " ")\n'

## Model Training

In [4]:
# 학습 데이터 선택
# 1. cbt_clean_10000_train.txt => 약 16만 문장
# 2. cbt_clean_20000_train.txt => 약 33만 문장
# 그 이상의 데이터는 위에서 추출 가능
selected_train_text = 'cbt_clean_20000_train.txt'

with io.open(selected_train_text, encoding='utf-8') as fin:
    texts = fin.read()

# Tokenize the text
sentences = nltk.sent_tokenize(texts)

print(f"sentences num: {len(sentences)}")

tokenized_texts = [list(map(str.lower, nltk.word_tokenize(sentence))) for sentence in sentences]

# word2Vec 모델 학습
model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=200,   # 임베딩 차원 수
    window=5,          # 컨텍스트 윈도우 크기
    min_count=5,       # 최소 단어 빈도 수
    workers=multiprocessing.cpu_count(),  # 병렬 처리 스레드 수
    sg=1,              # 1: Skip-gram, 0: CBOW
    epochs=10
)

sentences num: 337054


## Generate Results

In [7]:
def printSimilarity(word):
    sims = model.wv.most_similar(word, topn=10)
    return print(f'[{word}] : {sims}\n')

In [8]:
printSimilarity('queen')
printSimilarity('she')
printSimilarity('portrait')
printSimilarity('boy')
printSimilarity('royal')

[queen] : [('king', 0.4102681279182434), ('miranda', 0.4016765058040619), ('sheik', 0.3872550427913666), ('bitterness', 0.36715617775917053), ('conquest', 0.35870882868766785), ('mistveiled', 0.3586718440055847), ('displeases', 0.3546091318130493), ('approval', 0.3479524552822113), ('lowest', 0.34773555397987366), ('fanned', 0.3437991738319397)]

[she] : [('he', 0.7055046558380127), ('her', 0.7024463415145874), ('they', 0.5571861267089844), ('it', 0.5416773557662964), ('but', 0.5342990159988403), ('and', 0.5315732359886169), ('to', 0.5082858204841614), ('that', 0.501986026763916), ('as', 0.4880739152431488), ('.', 0.4869805872440338)]

[portrait] : [('painters', 0.41107723116874695), ('admirer', 0.3869825601577759), ('packet', 0.38291996717453003), ('snowflakes', 0.3622840344905853), ('bravery', 0.3588334023952484), ('coping', 0.34431248903274536), ('horserace', 0.33865538239479065), ('clearsighted', 0.3348265290260315), ('tinkled', 0.33473753929138184), ('pretense', 0.3338103592395782